# Baixa arquivos da conexão FTP do datasus para a pasta desejada do computador

In [3]:
import os.path
from ftplib import FTP


ftp = FTP('ftp.datasus.gov.br') # conecta no host
ftp.login() # login

     
ftp.cwd('/dissemin/publicos/CNES/200508_/Dados/ST/')# abre diretório desejado

print('Digite o ano dos dados do ST datasus que deseja baixar com dois dígitos. Ex: Para o ano de 2020, digite 20')
ano = str(input())

print('Digite o mes dos dados do ST datasus que deseja baixar com dois dígitos. Ex: Para o mes de Agosto, digite 08')
mes = str(input())


filenames = ftp.nlst('*'+ano+mes+'.DBF') # cria lista de arquivos desejados

savedir = 'C:/Users/Milena/Documents/Estudos/3M/dados' # cria variável de caminho para salvar os arquivos copiados do ftp
os.chdir(savedir)

# Faz loop sobre os a lista de arquivos e os baixa para a pasta desejada
for f in filenames:
    file = open(f, "wb")
    ftp.retrbinary('RETR '+ f, file.write)
    file.close() 
    print('Arquivo {} baixado para pasta'.format(f))
    
ftp.close() # fecha conexão    
print('Todos os arquivos desejados foram baixados./n A conexão ftp foi fechada') 

Digite o ano dos dados do ST datasus que deseja baixar com dois dígitos. Ex: Para o ano de 2020, digite 20
20
Digite o mes dos dados do ST datasus que deseja baixar com dois dígitos. Ex: Para o mes de Agosto, digite 08
06
Todos os arquivos desejados foram baixados./n A conexão ftp foi fechada


# Importa arquivos .DBF da pasta do computador, transforma em dataframe e empilha em um único dataframe

In [4]:
import os
import glob 
from pandas import DataFrame
from dbfread import DBF


os.chdir('C:/Users/Milena/Documents/Estudos/3M/dados')
list_of_files = glob.glob('*.DBF')

for idx, base in enumerate(list_of_files):
    if idx==0:
        dbf = DBF(base)
        frame = DataFrame(iter(dbf))
    else:
        base = DBF(base)  
        frame2 = DataFrame(iter(base))
        dbf = frame.append(frame2)
        
del(base)
del(idx)
del(list_of_files)

consolidado_csv = dbf.to_csv(mes+'_'+ano+'_'+'st_datasus.csv', sep= ';', encoding='latin1',index=False)     
        

In [16]:
dbf.shape

(1115, 201)

In [28]:
dbf.columns

Index(['CNES', 'CODUFMUN', 'COD_CEP', 'CPF_CNPJ', 'PF_PJ', 'NIV_DEP',
       'CNPJ_MAN', 'COD_IR', 'REGSAUDE', 'MICR_REG',
       ...
       'AP07CV01', 'AP07CV02', 'AP07CV05', 'AP07CV06', 'AP07CV03', 'AP07CV04',
       'ATEND_PR', 'DT_ATUAL', 'COMPETEN', 'NAT_JUR'],
      dtype='object', length=201)

# Inserção de dados consolidados em banco de dados Oracle
##### Exemplo de estrutura de inserção dos dados em banco de dados Oracle

In [ ]:
import pyodbc
import sqlalchemy
import time
import cx_Oracle as cx

oracle_connection_string = (
    'oracle+cx_oracle://{username}:{password}@') +
    cx.makedsn('{hostname}', {'port'}, service_name='{service_name}')

)
    
engine = a.create_engine(
    oracle_connection_string.format(
    username ='Insira o user name',
    password='Insira a senha',
    hostname='Insira o host name',
    port='Insira a porta',
    service_name='Insira a instancia',
    encoding='Insira o encoding',
    nencoding='Insira o nencoding'    
    )
)    
inicioinsert = time.time()
dbf.to_sql(
    name = 'tbl_st_datasus',
    con = engine,
    index = False,
    if_exists ='append'
)

fiminsert = time.time()

# mostra tempo que o insert no banco de dados levou
print('Insert na base de dados executado em:')
print(round(fiminsert-inicioinsert,2)) 